https://www.ft.dk/aktuelt/webtv/video/20222/salen/5.aspx?from=28-11-2022&to=19-12-2022&selectedMeetingType=Salen&committee=&as=1#player


In [1]:
import requests # Downloading webpages
from bs4 import BeautifulSoup # Extracting data from html files
import json
import pandas as pd
import re
from datetime import datetime
from moviepy.editor import VideoFileClip

In [2]:
url = "https://www.ft.dk/aktuelt/webtv/video/20222/salen/5.aspx?from=28-11-2022&to=19-12-2022&selectedMeetingType=Salen&committee=&as=1#player"
req = requests.get(url) # get the page

In [60]:
soup = BeautifulSoup(req.text, 'html.parser')

# Initialize lists to store the extracted data
names = []
texts = []
numbers = []

# Find all divs with class 'video-item-referat'
for div in soup.find_all("div", class_="video-item-referat"):
    # Extract name
    name_div = div.find("div", class_="name")
    name = name_div.get_text(strip=True)
    # Remove everything within parentheses and parentheses themselves
    name = re.sub(r'\([^)]*\)', '', name)
    # Remove "Formanden"
    name = name.replace("Formanden", "")
    # Remove whitespace at beginning and end
    name = name.strip()
    # Replace ø with oe
    name = name.replace("ø", "oe")

    # Extract texts from all <p> with class 'Tekst' and 'TekstIndryk'
    text_elements = div.find_all("p", class_=["Tekst", "TekstIndryk"])
    combined_text = ' '.join(p.get_text(strip=True) for p in text_elements)

    # Extract number (from id of 'video-item-content')
    content_div = div.find("div", class_="video-item-content")
    number = content_div['id'].split('_')[-1] if content_div and 'id' in content_div.attrs else 'N/A'

    # Append to lists
    names.append(name)
    texts.append(combined_text)
    numbers.append(number)

# Create DataFrame
df = pd.DataFrame({
    'Name': names,
    'Text': texts,
    'ID': numbers
})

# Remove rows that contain 'N/A'
df = df[df.ID != 'N/A']

In [61]:
df.head()

,Name,Text,ID
0,Soeren Gade,Der er som bekendt ingen minister på dette omr...,912053
1,Jens Joel,"Tak for det, formand. Og ja, det er rigtigt, a...",912064
2,Soeren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",912075
3,Thomas Danielsen,"Tak for det, formand. Venstre bifalder EU-refo...",912078
4,Soeren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",912079


In [62]:
script_tag = soup.find('script', text=re.compile('SpeachItem'))
# Extract all instances of SpeachItem
speach_items = re.findall(r'new SpeachItem\(.*\)', script_tag.string)
# Remove 'new SpeachItem'
speach_items = [re.sub(r'new SpeachItem', '', item) for item in speach_items]

In [63]:
# loop over speach_items and extract the data
start_times = []
end_times = []
start_times_clock = []
end_times_clock = []
numbers = []

video_start_clock = "09:59:41"
video_start_clock_datetime = datetime.strptime(video_start_clock, '%H:%M:%S')

for item in speach_items:
    speach_start_clock = item[19:27]
    speach_end_clock = item[41:49]

    # Convert to datetime
    speach_start_clock_datetime = datetime.strptime(speach_start_clock, '%H:%M:%S')
    speach_end_clock_datetime = datetime.strptime(speach_end_clock, '%H:%M:%S')

    # Calculate the timedelta (difference) from the video start
    speach_start_timestamp = (speach_start_clock_datetime - video_start_clock_datetime).total_seconds()
    speach_end_timestamp = (speach_end_clock_datetime - video_start_clock_datetime).total_seconds()

    # Append the original times and calculated seconds
    start_times_clock.append(speach_start_clock)
    end_times_clock.append(speach_end_clock)
    start_times.append(speach_start_timestamp)
    end_times.append(speach_end_timestamp)
    numbers.append(item[58:-2])

# Create DataFrame
time_stamps = pd.DataFrame({
    'Start': start_times,
    'End': end_times,
    'StartClock': start_times_clock,
    'EndClock': end_times_clock,
    'ID': numbers
})



In [64]:
time_stamps.head()

,Start,End,StartClock,EndClock,ID
0,70.0,107.0,10:00:51,10:01:28,912053
1,107.0,420.0,10:01:28,10:06:41,912064
2,420.0,428.0,10:06:41,10:06:49,912075
3,428.0,464.0,10:06:49,10:07:25,912078
4,464.0,475.0,10:07:25,10:07:36,912079


In [65]:
#combine the two dataframes on the number column
df = pd.merge(df, time_stamps, on='ID')
# drop the number column
df = df.drop(columns=['ID'])

In [66]:
# Save to csv
df.to_csv('speeches.csv', index=False)

In [67]:
df

,Name,Text,Start,End,StartClock,EndClock
0,Soeren Gade,Der er som bekendt ingen minister på dette omr...,70.0,107.0,10:00:51,10:01:28
1,Jens Joel,"Tak for det, formand. Og ja, det er rigtigt, a...",107.0,420.0,10:01:28,10:06:41
2,Soeren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",420.0,428.0,10:06:41,10:06:49
3,Thomas Danielsen,"Tak for det, formand. Venstre bifalder EU-refo...",428.0,464.0,10:06:49,10:07:25
4,Soeren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",464.0,475.0,10:07:25,10:07:36
5,Henrik Frandsen,"Mange tak, hr. formand. Jeg vil gerne starte m...",475.0,574.0,10:07:36,10:09:15
6,Soeren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",574.0,589.0,10:09:15,10:09:30
7,Karina Lorentzen Dehnhardt,"Det her er jo et af de forslag, som der, efter...",589.0,628.0,10:09:30,10:10:09
8,Soeren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",628.0,646.0,10:10:09,10:10:27
9,Peter Skaarup,Jeg skal også på Danmarksdemokraternes vegne t...,646.0,705.0,10:10:27,10:11:26


# TRIM

In [68]:
def trim_video(video_file_path, start_time, end_time, trimmed_video_path):
    video = VideoFileClip(video_file_path).subclip(start_time, end_time)
    video.write_videofile(trimmed_video_path, codec="libx264", audio_codec="aac")

In [69]:
# Path to your MP4 video file
video_file_path = '../data/video/VOD_08-12-2022_M__de_i_salen.mp4'
# loop over rows in df and trim and save video
for index, row in df.iterrows():
    # Create filename
    filename = f"{index}_{row['Name']}.mp4"
    # Create path
    path = f"../data/test/clips/video1/{filename}"
    # Trim video
    trim_video(video_file_path, row['Start'], row['End'], path)

                                                                
                                                                
t:   0%|          | 2/925 [11:40<89:48:48, 350.30s/it, now=None]

Moviepy - Building video ../data/test/clips/video1/0_Soeren Gade.mp4.
MoviePy - Writing audio in 0_Soeren GadeTEMP_MPY_wvf_snd.mp4













                                                                
                                                                
t:   0%|          | 2/925 [11:41<89:56:34, 350.81s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ../data/test/clips/video1/0_Soeren Gade.mp4


























                                                                
                                                                
t:   0%|          | 2/925 [11:44<90:17:11, 352.15s/it, now=None]

...

Moviepy - Building video ../data/test/clips/video1/40_Soeren Gade.mp4.
MoviePy - Writing audio in 40_Soeren GadeTEMP_MPY_wvf_snd.mp4









                                                                 
                                                                 
t:   0%|          | 2/925 [14:27<111:09:52, 433.58s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ../data/test/clips/video1/40_Soeren Gade.mp4


















                                                                 
                                                                 
t:   0%|          | 2/925 [14:28<111:23:26, 434.46s/it, now=None]

Moviepy - Done !
Moviepy - video ready ../data/test/clips/video1/40_Soeren Gade.mp4
